In [1]:
# | default_exp unetr3d_decoder

# Imports

In [2]:
# | export

import torch

from torch import nn

# Architecture

### Basic block

In [3]:
# | export


class UNetR3DBlock(nn.Module):
    def __init__(self, in_dim, out_dim, conv_kernel_size, deconv_kernel_size, is_first_layer):
        super().__init__()

        self.conv = nn.Conv3d(
            in_dim, in_dim, kernel_size=conv_kernel_size, padding=tuple([k // 2 for k in conv_kernel_size])
        )
        if not is_first_layer:
            in_dim = in_dim * 2
        self.deconv = nn.ConvTranspose3d(
            in_dim, out_dim, kernel_size=deconv_kernel_size, stride=deconv_kernel_size, padding=0
        )

    def forward(self, current_layer, previous_layer=None):
        x = self.conv(current_layer)
        if previous_layer is not None:
            x = torch.cat([x, previous_layer], dim=1)
        x = self.deconv(x)
        return x

In [4]:
test = UNetR3DBlock(768, 192, (3, 3, 3), (2, 2, 2), is_first_layer=True)
display(test)
display(test(torch.randn(2, 768, 4, 8, 8)).shape)


UNetR3DBlock(
  (conv): Conv3d(768, 768, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (deconv): ConvTranspose3d(768, 192, kernel_size=(2, 2, 2), stride=(2, 2, 2))
)

torch.Size([2, 192, 8, 16, 16])

In [5]:
test = UNetR3DBlock(192, 48, (3, 3, 3), (2, 2, 2), is_first_layer=False)
display(test)
display(test(torch.randn(2, 192, 8, 16, 16), torch.randn(2, 192, 8, 16, 16)).shape)


UNetR3DBlock(
  (conv): Conv3d(192, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (deconv): ConvTranspose3d(384, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
)

torch.Size([2, 48, 16, 32, 32])

### Complete decoder

In [6]:
# | export


class UNetR3DDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.blocks = nn.ModuleList()
        for i in range(len(config["stages"])):
            stage = config["stages"][-i - 1]

            in_dim = stage["_out_dim"]
            is_first_layer = i == 0

            if i == len(config["stages"]) - 1:
                out_dim = config["decoder"]["num_outputs"]
                deconv_kernel_size = stage["_out_patch_size"]
            else:
                out_dim = stage["_in_dim"]
                deconv_kernel_size = tuple([o // i for o, i in zip(stage["_out_patch_size"], stage["_in_patch_size"])])

            self.blocks.append(
                UNetR3DBlock(
                    in_dim=in_dim,
                    out_dim=out_dim,
                    conv_kernel_size=config["decoder"]["conv_kernel_size"],
                    deconv_kernel_size=deconv_kernel_size,
                    is_first_layer=is_first_layer,
                )
            )

    def forward(self, embeddings):
        # embeddings is a list of (B, C_layer, D_layer, W_layer, H_layer)
        embeddings = embeddings[::-1]

        decoded = None
        for i in range(len(embeddings)):
            embedding = embeddings[i]
            if i == 0:
                decoded = self.blocks[i](embedding)
            else:
                decoded = self.blocks[i](embedding, decoded)

        return decoded

In [7]:
test_config = {
    "decoder": {
        "conv_kernel_size": (3, 3, 3),
        "num_outputs": 5,
    },
    "stages": [
        {
            "_in_dim": 12,
            "_in_patch_size": (1, 4, 4),
            "_out_dim": 12,
            "_out_patch_size": (1, 4, 4),
        },
        {
            "_in_dim": 12,
            "_in_patch_size": (1, 4, 4),
            "_out_dim": 48,
            "_out_patch_size": (2, 8, 8),
        },
        {
            "_in_dim": 48,
            "_in_patch_size": (2, 8, 8),
            "_out_dim": 192,
            "_out_patch_size": (4, 16, 16),
        },
        {
            "_in_dim": 192,
            "_in_patch_size": (4, 16, 16),
            "_out_dim": 768,
            "_out_patch_size": (8, 32, 32),
        },
    ],
}

test = UNetR3DDecoder(test_config)
display(test)
display(
    test(
        [
            torch.randn(2, 12, 32, 64, 64),
            torch.randn(2, 48, 16, 32, 32),
            torch.randn(2, 192, 8, 16, 16),
            torch.randn(2, 768, 4, 8, 8),
        ]
    ).shape
)


UNetR3DDecoder(
  (blocks): ModuleList(
    (0): UNetR3DBlock(
      (conv): Conv3d(768, 768, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (deconv): ConvTranspose3d(768, 192, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (1): UNetR3DBlock(
      (conv): Conv3d(192, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (deconv): ConvTranspose3d(384, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (2): UNetR3DBlock(
      (conv): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (deconv): ConvTranspose3d(96, 12, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    )
    (3): UNetR3DBlock(
      (conv): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (deconv): ConvTranspose3d(24, 5, kernel_size=(1, 4, 4), stride=(1, 4, 4))
    )
  )
)

torch.Size([2, 5, 32, 256, 256])

In [8]:
from neuro_utils.describe import describe_model

describe_model(test)

/home/users/arjun.agarwal/miniconda3/lib/python3.9/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Total Parameters: 18,326,189
+------------------------+------------+
|         Module         | Parameters |
+------------------------+------------+
|  blocks.0.conv.weight  | 15,925,248 |
|   blocks.0.conv.bias   |    768     |
| blocks.0.deconv.weight | 1,179,648  |
|  blocks.0.deconv.bias  |    192     |
|  blocks.1.conv.weight  |  995,328   |
|   blocks.1.conv.bias   |    192     |
| blocks.1.deconv.weight |  147,456   |
|  blocks.1.deconv.bias  |     48     |
|  blocks.2.conv.weight  |   62,208   |
|   blocks.2.conv.bias   |     48     |
| blocks.2.deconv.weight |   9,216    |
|  blocks.2.deconv.bias  |     12     |
|  blocks.3.conv.weight  |   3,888    |
|   blocks.3.conv.bias   |     12     |
| blocks.3.deconv.weight |   1,920    |
|  blocks.3.deconv.bias  |     5      |
+------------------------+------------+


# nbdev

In [9]:
!nbdev_export